In [1]:
# Dependency Graph
import networkx as nx
import matplotlib.pyplot as plt

courses = ['111','120','124','125','126','307','308','309','324']
g = nx.DiGraph()
g.add_nodes_from(courses)

# 0 1 2 3 4 5 6 7 8
g.add_edge(courses[1],courses[2])
g.add_edge(courses[2],courses[3])
g.add_edge(courses[3],courses[4])
g.add_edge(courses[3],courses[5])
g.add_edge(courses[4],courses[6])
g.add_edge(courses[5],courses[7])
g.add_edge(courses[6],courses[7])
g.add_edge(courses[4],courses[8])

nx.draw(g,with_labels=True)
plt.draw()
# plt.show()


In [2]:
# Parse data
# 2008-2018 for 2019 prediction
# i.e. 2008-2017 for 2018 validation
# i.e. 2008-2016 for 2017 validation
yFrom = 2008
yTo = 2015


import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import csv
 
# df = pd.read_excel('File.xlsx', sheetname='Sheet1')
# print("Column headings:")
# print(df.columns)

mydict = {} 
# num -> year -> percentage of enrollment

mydict2 = {}
# num -> year -> time -> percentage of enrollment

# ['Yr,Crs No,Section,Current Enrlmnt,Percentageof Enrollment,Limit/Est Enrlmnt,Start Time,End Time']
# 2008-2018
# with open('project/su-1.csv', mode='r') as infile:
#2008-2017
# with open('project/data_filtered_2008-2017.csv', mode='r') as infile:
#2008-2016
with open('../data/data_filtered_' + str(yFrom) + '-' + str(yTo) + '.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter='\t')
#     with open('coors_new.csv', mode='w') as outfile:
#         writer = csv.writer(outfile)

    i = 0
    for row in reader:
        if i == 0:
            i += 1
            continue
        
        tokens = row[0].split(",")
#         print(tokens)
        
        courseNo = tokens[1]
#         print(courseNo)
        year = tokens[0]
        perEnroll = tokens[4]
        
        secDict = {}
        if courseNo not in mydict.keys():
            allEnrolls = []
            allEnrolls.append(perEnroll)
            secDict[year] = allEnrolls
            mydict[courseNo] = secDict
#             print(mydict)
        else:
            secDict = mydict[courseNo]
#             print(secDict)
            allEnrolls = []
            if year in secDict.keys():
                allEnrolls = secDict[year]
            allEnrolls.append(perEnroll)
            
            secDict[year] = allEnrolls
#             print(secDict)
            mydict[courseNo] = secDict
#             print(mydict)
    
# print(mydict)
# del mydict[""]
import json
with open('../data/data.json', 'w') as outfile:
    json.dump(mydict, outfile)



In [3]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import csv
 
# df = pd.read_excel('File.xlsx', sheetname='Sheet1')
 
# print("Column headings:")
# print(df.columns)

mydict2 = {}
# num -> year -> time -> percentage of enrollment


# ['Yr,Crs No,Section,Current Enrlmnt,Percentageof Enrollment,Limit/Est Enrlmnt,Start Time,End Time']
# 2008-2018
# with open('project/su-1.csv', mode='r') as infile:
# 2008-2017
# with open('project/data_filtered_2008-2017.csv', mode='r') as infile:
# 2008-2016
with open('../data/data_filtered_' + str(yFrom) + '-' + str(yTo) + '.csv', mode='r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    
    i = 0
    for row in reader:
        if i == 0:
            i += 1
            continue

        tokens = row[0].split(",")
    #         print(tokens)

        courseNo = tokens[1]
    #         print(courseNo)
        year = tokens[0]
        perEnroll = tokens[4]
        time = tokens[6]

        secDict = {} # year-> time -> percent. enrollment
        thirdDict = {} # time -> percent. enrollment
        if courseNo not in mydict2.keys():

            allEnrolls = []
            allEnrolls.append(perEnroll)
            thirdDict[time] = allEnrolls
            secDict[year] = thirdDict
            mydict2[courseNo] = secDict
    #             print(mydict2)
        else:
            secDict = mydict2[courseNo]
    #             print(secDict)
            allEnrolls = []
            thirdDict = {}
            if year in secDict.keys():
                thirdDict = secDict[year]

            if time in thirdDict:
                allEnrolls = thirdDict[time]

            allEnrolls.append(perEnroll)
            thirdDict[time] = allEnrolls
            secDict[year] = thirdDict
    #             print(secDict)
            mydict2[courseNo] = secDict
    #             print(mydict)
    
# print(mydict)
# del mydict2[""]
import json
with open('../data/data2.json', 'w') as outfile:
    json.dump(mydict2, outfile)

In [4]:
# tables version 1.0 
# with only percent. enrollment

from tabulate import tabulate

for num in mydict.keys():
    secDictionary = mydict[num]
    line = []
    maxHeader = 1
    for year in sorted(secDictionary.keys()):
        row = []
        row.append(year)
        for enroll in secDictionary[year]:
            newEnroll = "%.2f" % float(enroll)
            if float(enroll) >= 57 and float(enroll) <= 80:
                newEnroll += "(\u2713)"
            else:
                newEnroll += "(\u2717)"
            row.append(newEnroll)
        line.append(row)
        if len(secDictionary[year]) > maxHeader:
            maxHeader = len(secDictionary[year])
        
    headers = []
    headers.append("year")
    for i in range(maxHeader):
        headers.append("Percent. Enrollment sec %d" % (i+1))
    
#     print("Math " + num)
#     print (tabulate(line, headers=headers))
#     print()
    

In [5]:
# tables version 2.0
# with percent. enroll + time slot
from tabulate import tabulate

for num in mydict2.keys():
    secDictionary = mydict2[num]
    line = []
    maxHeader = 1
    # year -> time -> percent. enrollment
    for year in sorted(secDictionary.keys()):
        thirdDict = secDictionary[year]
        row = []
        row.append(year)
        # time -> percent. enrollment
        for time in thirdDict.keys():
            enrolls = thirdDict[time]
            newEnrolls = ""
            for enroll in enrolls:
                newEnroll = "%.2f" % float(enroll)
                if float(enroll) >= 57 and float(enroll) <= 80:
                    newEnroll += "(" + time + "\u2713)"
                else:
                    newEnroll += "(" + time + "\u2717)"
                newEnrolls += " " + newEnroll
            row.append(newEnrolls.strip())


#         print(thirdDict)
        if len(secDictionary[year]) > maxHeader:
            maxHeader = len(secDictionary[year])
        line.append(row)
        
    headers = []
    headers.append("year")
    for i in range(maxHeader):
        headers.append("Percent. Enrollment sec %d" % (i+1))
    
#     print("Math " + num)
#     print (tabulate(line, headers=headers))
#     print()
    

In [6]:
# tables 3.0
# with percent. enroll + time slots being headers
from tabulate import tabulate

for num in mydict2.keys():
    secDictionary = mydict2[num]
    line = []
    maxHeader = 1
    timeHeaders = set()
    
    # year -> time -> percent. enrollment
    for year in sorted(secDictionary.keys()):
        thirdDict = secDictionary[year]
        # time -> percent. enrollment
        for time in sorted([int(i) for i in thirdDict.keys()]):
            timeHeaders.add(time)
    
    def sortByTime(timeHeaders):
        timeHeaders = sorted(list(timeHeaders))
        timeHeaders = [str(i) for i in timeHeaders]
        for time in timeHeaders:
            if time == "110":
                if timeHeaders[1] == "220":
                    t110 = timeHeaders[0]
                    t220 = timeHeaders[1]
                    timeHeaders.remove(t110)
                    timeHeaders.remove(t220)
                    timeHeaders.append(t110)
                    timeHeaders.append(t220)
#                     print(timeHeaders)
                    break
                t110 = timeHeaders[0]
                timeHeaders.remove(t110)
                timeHeaders.append(t110)
                break
        return timeHeaders
        
#         print(timeHeaders)
        
    timeHeaders = sortByTime(timeHeaders)

    
    # year -> time -> percent. enrollment
    for year in sorted(secDictionary.keys()):
        thirdDict = secDictionary[year]
        row = []
        row.append(year)
        # time -> percent. enrollment
#       int list -> sorted str list
        times = sortByTime([int(i) for i in thirdDict.keys()])
        for time in timeHeaders:
            if time not in times:
                row.append("")
            else:
                enrolls = thirdDict[time]
                newEnrolls = ""
                for enroll in enrolls:
                    newEnroll = "%.2f" % float(enroll)
                    if float(enroll) >= 57 and float(enroll) <= 80:
#                         newEnroll += "(" + time + "\u2713)"
                        newEnroll += "(\u2713)"
                        # for latex
#                         newEnroll += "(" + r"\cmark" + ")"
                    else:
#                         newEnroll += "(" + time + "\u2717)"
                        newEnroll += "(\u2717)"
                        # for latex
#                         newEnroll += "(" + r"\xmark" + ")"
                    newEnrolls += " " + newEnroll
                row.append(newEnrolls.strip())
#         print(row)


#         print(thirdDict)
        if len(secDictionary[year]) > maxHeader:
            maxHeader = len(secDictionary[year])
        line.append(row)
        
    headers = []
    headers.append("year")
    for i in timeHeaders:
#         headers.append("Percent. Enrollment sec %d" % (i+1))
        if i == "940" or i == "1050" or i == "1200": 
            headers.append(i + " am")
        if i == "110" or i == "220":
            headers.append(i + " pm")
#     print("\\textbf{Math " + num + "}")
    print("Math" + num)
    print (tabulate(line, headers=headers))
#     format="latex_raw"
    print()

Math111
  year  940 am    1050 am
------  --------  ---------
  2008  45.71(✗)  60.00(✓)
  2009            65.71(✓)
  2010            82.50(✗)
  2011            77.14(✓)
  2012            95.00(✗)
  2013            67.50(✓)
  2014            77.50(✓)
  2015            70.00(✓)

Math120
  year  940 am    1050 am    1200 am
------  --------  ---------  ---------
  2008  71.43(✓)  57.14(✓)   77.14(✓)
  2009  88.00(✗)  72.00(✓)   104.00(✗)
  2010            74.29(✓)   60.00(✓)
  2011            85.71(✗)   97.14(✗)
  2012  80.00(✓)  42.86(✗)   74.29(✓)
  2013            62.86(✓)   63.33(✓)
  2014            28.57(✗)   74.29(✓)
  2015                       91.43(✗)

Math124
  year  940 am     1050 am    1200 am    110 pm
------  ---------  ---------  ---------  --------
  2008  62.86(✓)              77.14(✓)   51.43(✗)
  2009  62.86(✓)   85.71(✗)   71.43(✓)   40.00(✗)
  2010  71.43(✓)   57.14(✓)   74.29(✓)   94.29(✗)
  2011  70.00(✓)   76.67(✓)   80.00(✓)   53.33(✗)
  2012  100.00(✗)  85.71(

In [7]:
# For num of sections

count = {}
# course -> year -> # sections

# num -> year -> time -> percentage of enrollment
for i in mydict2.keys():
    years = mydict2[i]
    # year -> time -> percentage of enrollment
    for y in years.keys():
        times = years[y]
        
        secCount = 0
        # time -> percentage of enrollment
        for t in times.keys():
            enrolls = times[t]
            secCount += len(enrolls)
        
        yCount = {}
        if i in count:
            yCount = count[i]
        
        yCount[y] = secCount
        count[i] = yCount

# print(count)







In [8]:
# Print count

lines = []
rows = []
# course -> year -> # sections
mydict3 = {}

for num in sorted(count):
    years = count[num]
    line = []
    temp = {}
    for y in sorted(years):
        row = []
        row.append(y)
        nSections = years[y]
        row.append(nSections)
        line.append(row)
        
        if num in mydict3:
            temp = mydict3[num]
        temp[y] = nSections

    mydict3[num] = temp
    
    headers = []
    headers.append("year")
    for i in range(1):
        headers.append("# sections")
    
#     print("Math " + num)
#     print (tabulate(line, headers=headers))
#     print()

# print(mydict3)
# print(count)



In [9]:
# year -> course -> # sections
mydict4 = {}
for num in mydict3:
    years = mydict3[num]
    
    for y in years:
        nSec = years[y]
        temp = {}
        if y in mydict4:
            temp = mydict4[y]
            
        temp[num] = nSec
        mydict4[y] = temp
        
# print(mydict4)

line = []
for y in sorted(mydict4):
    course = mydict4[y]

    row = []
    row.append(y)
    for i in sorted(course):
        nSections = course[i]
        row.append(nSections)
    
    line.append(row)
    
headers = []
headers.append("year")
for i in mydict4['2008'].keys():
    headers.append("Math " + str(i) + " # sections")

print("Math " + num)
print (tabulate(line, headers=headers))
print()

# f = open("table_#sections.txt", "w")
# f.write(tabulate(line, headers=headers))

Math 324
  year    Math 111 # sections    Math 120 # sections    Math 124 # sections    Math 125 # sections    Math 126 # sections    Math 307 # sections    Math 308 # sections    Math 309 # sections    Math 324 # sections
------  ---------------------  ---------------------  ---------------------  ---------------------  ---------------------  ---------------------  ---------------------  ---------------------  ---------------------
  2008                      2                      3                      3                      4                      4                      3                      5                      1                      4
  2009                      1                      3                      4                      4                      4                      4                      6                      1                      4
  2010                      1                      2                      4                      4                      5              

In [10]:
# please run only once
# reinitialize count again o/w
import math

s = {}
# course -> year -> appro. # sections

# count
# course -> year -> # sections

y2018 = {}

# course i -> year j -> appropriate # sections
for i in count.keys():
    years = count[i]
    years2 = mydict[i]
    temp = {}
#     print(years)
    for y in years:
        nSec = years[y]
        enrolls = years2[y]
        if y == '2018':
            y2018[i] = nSec
        
        c2Up = 0
        c2Lo = 0
        c1Up = 0
        c1Lo = 0
#         print(enrolls)
        for enroll in enrolls:
            enroll = float(enroll)
            if enroll >= 80 and enroll < 100:
                if len(enrolls) == 1:   
                    c1Up += 1
                else:
                    c2Up += 1
            elif enroll >= 29 and enroll < 57:
                c2Lo += 1
            elif enroll >= 100:
                c1Up += 1
            elif enroll < 29:
                c1Lo += 1
        if i in count:
            temp = count[i]
        temp[y] = nSec + 1 * float(c2Up) / 2.0 - 1 * float(c2Lo) / 2.0 + 1 * c1Up - 1 * c1Lo
        temp[y] = math.ceil(temp[y])
        if temp[y] - nSec > 1:
            temp[y] = nSec + 1
        elif nSec - temp[y] > 1:
            temp[y] = nSec - 1
        if temp[y] < 1:
            temp[y] = 1
        
#         print(nSec)
#         print(enrolls)
#         print(temp[y])
        
    s[i] = temp
# print(s)

for i in s:
    
    headers = ['Year', 'Appr. # sections']
    data = sorted([(k,v) for k,v in s[i].items()]) # flip the code and name and sort
    print("Math" + i)
    print(tabulate(data, headers=headers))
    print()
    
with open('table_' + str(yFrom) + '-' + str(yTo) + 'appro#sections.txt', 'w+') as f:
    for i in s:
        headers = ['Year', 'Appr. # sections']
        data = sorted([(k,v) for k,v in s[i].items()]) # flip the code and name and sort
        f.write("Math" + i + "\n")
        f.write(tabulate(data, headers=headers))
        f.write("\n\n")

Math111
  Year    Appr. # sections
------  ------------------
  2008                   2
  2009                   1
  2010                   2
  2011                   1
  2012                   2
  2013                   1
  2014                   1
  2015                   1

Math120
  Year    Appr. # sections
------  ------------------
  2008                   3
  2009                   4
  2010                   2
  2011                   3
  2012                   3
  2013                   2
  2014                   1
  2015                   2

Math124
  Year    Appr. # sections
------  ------------------
  2008                   3
  2009                   4
  2010                   5
  2011                   4
  2012                   5
  2013                   3
  2014                   4
  2015                   4

Math125
  Year    Appr. # sections
------  ------------------
  2008                   3
  2009                   5
  2010                   5
  2011              

In [11]:
# weight

# Automation DONE
# 2008-2018
# weigh = [float(x)/10 for x in range(1, 7)]
# weigh.extend([0.8, 1.0, 1.2, 1.4, 1.6])
# 2008-2017
# weigh = [float(x)/10 for x in range(1, 6)]
# weigh.extend([0.7, 0.9, 1.1, 1.3, 1.5])
# 2008-2016
rangeTotal = yTo - yFrom + 1
weigh = []
if rangeTotal > 5:
    weigh = [float(x)/10 for x in range(1, rangeTotal - 5 + 1)]
last = weigh[len(weigh) - 1]
weigh.extend([last + 0.2 * i for i in range (1, rangeTotal - len(weigh) + 1)])

# w = [float(x) / 10 / sum(weigh) for x in range(1, 7)]
# w.extend([0.8 /sum(weigh), 1.0 / sum(weigh), 1.2 / sum(weigh), 1.4 / sum(weigh), 1.6 /sum(weigh)])
# w = [float(x) / 10 / sum(weigh) for x in range(1, 6)]
# w.extend([weigh[5] /sum(weigh), weigh[6] / sum(weigh), weigh[7] / sum(weigh), weigh[8] / sum(weigh), weigh[9] /sum(weigh)])
w = [float(weigh[i]) / sum(weigh) for i in range(len(weigh))]

# print(sum(weigh))
# print(w)

# s = {}
# course -> year -> appro. # sections

S = {}
# course -> appro.# sections in 2019


for i in s:
    years = s[i]
    
    appSec = 0
    j = 0
#     print(years)
    for y in sorted(years):
        appYSec = years[y]
#         print(appYSec)
#         print(w[j])
        appSec += float(w[j]) * float(appYSec)
#         if i == '111':
#             print(appSec)
        j += 1
#         print(j)
    
#     if appSec < 1:
#         appSec = 1
    S[i] = "%.2f" % appSec
# print(S)
# print(y2018)


headers = ['Couse No.', 'Appr. # sections', '2018 # sections']
data = sorted([[k,v + " (" + u"\u2248" + str(int(float(v))) + ")"] for k,v in S.items()]) # flip the code and name and sort

added = False
j = 0
for i in y2018:
    for k in data:
        if not added:
            added = True
            data[j].append(y2018[i])
            j += 1
    added = False

title = ""
if yTo+1 == 2019:
    title += "Predict "
else:
    title += "Validate "
title += str(yTo + 1)
print(title)
print(tabulate(data, headers=headers, numalign="left"))
print()

with open("table_" + str(yTo+1) + 'appro#sections.txt', "w") as f:
    f.write (tabulate(data, headers=headers, numalign="left"))
    
import csv

headers2 = ['Course', '# Sections']
data2 = sorted([[k,str(int(float(v)))] for k,v in S.items()]) # flip the code and name and sort
          
with open("table_" + str(yTo+1) + '_csv.txt', "w") as f:
    f.write(tabulate(data2, headers=headers2, numalign="Left"))
    
        
# import csv

# with open('table_csv.csv', 'rb') as f:
#   data = list(csv.reader(f))

Validate 2016
Couse No.    Appr. # sections
-----------  ------------------
111          1.22 (≈1)
120          2.12 (≈2)
124          4.00 (≈4)
125          5.24 (≈5)
126          6.82 (≈6)
307          6.12 (≈6)
308          6.33 (≈6)
309          3.27 (≈3)
324          4.82 (≈4)



In [12]:
# Stage I result


print(title)
# tabulate to latex
# tablefmt="latex"
print (tabulate(data2, headers=headers2))
print()

Validate 2016
  Course    # Sections
--------  ------------
     111             1
     120             2
     124             4
     125             5
     126             6
     307             6
     308             6
     309             3
     324             4



In [13]:
# from pandas import Timestamp
# from ggplot import *
# %matplotlib inline
# df = pd.DataFrame({headers[i] : data2[i] for i in range(len(headers2))})
# p = ggplot(aes('Couse No.'), data=df)
# p + geom_histogram(binwidth=1)